# Second Assignment of RTII course

## User Interface node

In this Jupyter notebook I have implemented a user interface useful to control the behavior of a robot, within  a simulated environment. Moreover the  node communicates with three different nodes:

1. nodes go_to_point.py 
2. state_machine.cpp 
3. position_service.cpp 

They all are available within the **action** branch belonging to the package rt2_assignment1 
Inside this notebook, different widgets and visualization methods have been employed to directly interact  with the robot simulation launched within Gazebo.


## How to run the code

Within a new shell, please type:

roslaunch rt2_assignment1 jupisim.launch

Then open this jupiter notebook by running:

jupyter notebook --allow-root --ip 0.0.0.0




Here Below, needed headers are imported and global variables are initialized 

In [ ]:

from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
import time
import math
from rt2_assignment1.srv import Command
import actionlib
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from matplotlib import animation, rc
import ipywidgets as widgets
from IPython.display import display
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from nav_msgs.msg import Odometry
from rt2_assignment1.msg import GoalReachingActionResult
%matplotlib widget

# initializing the node 
rospy.init_node('user_interface')
# user_interface client for starting the  "go to point behaviour"
ui_client = rospy.ServiceProxy('/user_interface', Command)
# action client to cancel the action goal
client = actionlib.SimpleActionClient('go_to_point', rt2_assignment1.msg.GoalReachingAction)
# declaring a publisher of /cmd_vel topic for controlling the robot's velocity
pub=rospy.Publisher('/cmd_vel', Twist, queue_size=1)
time.sleep(10)

# global variables initialisation
cancelled=0 
reached=0 
active=False
start_time=0
stop_time=0
data = list()




## Reaching goal behavior 

```go_to_point``` behavior can be activated and deactivated by user clicking on START or STOP button. According to the```on_click``` method, each time a button is clicked, the clbk is executed. 
For this reason the service client is called inside the callback of the start button. On the other hand, inside the callback of the stop button, the robot is at first stopped publishing a zero velocity on ```/cmd_vel```. Then action client is called so that all the goals could be cancelled 

In [ ]:

button_start = widgets.Button(description="START",layout=widgets.Layout(widht='auto',grid_area='b1'),
                style=widgets.ButtonStyle(button_color='green') )

button_stop = widgets.Button(description="STOP",  layout=widgets.Layout(width='auto', grid_area='b2'),
                    style=widgets.ButtonStyle(button_color='red'))



def clicked_start(b): 
     global start_time
     global active
     global count
     #call the service client
     ui_client("start")      
     active=True
     take=False
     start_time=time.time()
     
     
        
def clicked_stop(b):
     global cancelled
     global stop_time
     global active
     global data
     # call the action client
     client.cancel_all_goals()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y = 0
     twist_msg.angular.z = 0
     #publish a zero velocity
     pub.publish(twist_msg)
     ui_client("stop")
     if active==True:
         cancelled = cancelled + 1 
         active=False
         stop_time=time.time()
         data.append(stop_time-start_time)
 
# on click method

button_start.on_click(clicked_start)
button_stop.on_click(clicked_stop)

In [ ]:
# GridBox is used for displaying buttons 
widgets.GridBox(children=[button_start,button_stop],
                layout=widgets.Layout(width='80%',grid_template_rows='auto auto auto',
                grid_template_columns='16% 16% 16% 16% 16% 16% ',
                grid_template_areas='''"b1 b2 . . . "''')
               )

The callback of  ```/go_to_point/result``` subscriber, is executed only once a goal is reached. 
Then some variables exploited for visualization purposes are updated

In [ ]:
def reached_goal_clb(msg):
  global reached
  global start_time,stop_time
  global data 
  stop_time=time.time()
  data.append(stop_time-start_time)
  start_time=time.time()
  reached = reached+1
  

jr.subscribe('/go_to_point/result',GoalReachingActionResult, reached_goal_clb)

## Set velocity through sliders

Here below are inserted two **sliders**. The user can chose a linear or angular velocity between -1 and 1. Then, the chosen value is directly published on ```/cmd_vel```, soo that the robot could be piloted 

In [ ]:
msg = Twist()

a = widgets.FloatSlider(description='Linear Velocity' ,min=-1.0, max=1.0)
a.style.handle_color = 'salmon'
display(a)

b = widgets.FloatSlider(description='Angular Velocity',min=-1.0, max=1.0)
b.style.handle_color = 'salmon'
display(b)


def on_value_change(change):
    global msg
    msg.linear.x = change['new']
    pub.publish(msg)
    
def on_valueang_change(change):
    global msg
    msg.angular.z = change['new']
    pub.publish(msg)

a.observe(on_value_change, names='value')
b.observe(on_valueang_change, names='value')

## Pilot the robot

The user will be able to control the robot through the five buttons reported below. Then, Inside the buttons clbk:
1. stop function is called for stopping the ```go_to_point``` behavior, and **cancel** goals if any. 
2. for each button specific Twist message is published, according to user's preferences 

In [ ]:

BF = widgets.Button(description="GO FORWARD",layout=Layout(width='auto', grid_area='BF'),
style=ButtonStyle(button_color='yellow'))

BL = widgets.Button(description="TURN LEFT",layout=Layout(width='auto', grid_area='BL'),
style=ButtonStyle(button_color='yellow'))

BB = widgets.Button(description="GO BACKWARD",layout=Layout(width='auto', grid_area='BB'),
style=ButtonStyle(button_color='yellow'))

BR = widgets.Button(description="TURN RIGHT",layout=Layout(width='auto', grid_area='BR'),
style=ButtonStyle(button_color='yellow'))

BS = widgets.Button(description="STOP",layout=Layout(width='auto', grid_area='BS'),
style=ButtonStyle(button_color='red'))



def stop():
     global cancelled
     global stop_time
     global active
     global data
     client.cancel_all_goals()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y = 0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
     ui_client("stop")
     if active==True:
         cancelled = cancelled + 1 
         active=False
         stop_time=time.time()
         data.append(stop_time-start_time)
        
def click_BF(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 1
     twist_msg.linear.y=0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
        
def click_BL(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y=0
     twist_msg.angular.z = 1
     pub.publish(twist_msg)
        
def click_BR(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y=0
     twist_msg.angular.z = -1
     pub.publish(twist_msg)
        
def click_BB(f):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = -1
     twist_msg.linear.y=0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
    
def click_BS(b):
     stop()
     twist_msg = Twist()
     twist_msg.linear.x = 0
     twist_msg.linear.y=0
     twist_msg.angular.z = 0
     pub.publish(twist_msg)
    

            
            
BF.on_click(click_BF)
BR.on_click(click_BR)
BB.on_click(click_BB)
BL.on_click(click_BL)
BS.on_click(click_BS)


In [ ]:
widgets.GridBox(children=[BF, BB, BL, BR ,BS],
                layout=widgets.Layout(width='80%',grid_template_rows='auto auto auto',
                grid_template_columns='16% 16% 16% 16% 16% 16% ',
                grid_template_areas='''". .  . BF  . ."". . BL BS BR . "" . .  . BB . . "''')
               )

## Barplot visualization

The barplot is a powerful tool, useful for visualizing the **reached** and the **deleted goals**. As far as the reached variable is concerned, it is updated each time a message gets published on ```/go_to_point/result```. Instead, the ```cancelled``` variable, is updated if the user decides to terminate the reaching goal behavior.

In [ ]:
y = (reached,cancelled)
index = np.arange(2)

f1,axis2= plt.bar(index,y,width=0.5,color='r')
plt.ylabel('Values')
plt.title('Targets')
plt.xticks(index, ('reached', 'cancelled'))
plt.show()

## Histogram visualization

The histogram representation is here exploited for visualizing the time employed by the robot to reach a goal. Moreover, ```start_time``` and ```stop_time``` are variables used to generate a timer useful for computing the length of each bar. To conclude with, inside the clbk of ```/go_to_point/result``` a data.append is performed, in order to continuously update the histogram, as new targets are reached.

In [ ]:
X=list(range(0,len(data),1))
fig2=plt.subplot(111)
plt.bar(X,data,width=0.5,color='b')
plt.ylabel('time (s)')
plt.title('Mean time to reach a target')
plt.show()

## Line plots and animator

For visualizing changes in data, the ```animation.FuncAnimation``` method, has been employed.
Here below, it is evident that Data continuously asking for subscribtion inside ```cmd_vel``` and ```odom clbk``` was updated and inserted in different arrays. 
> Every time the notebook ```Run``` button is clicked, the Figure gets refreshed inside the callback of the animator.

## Linear Velocity

In the figure below, it is possible to visualize the linear velocity subscribed by ```/odom``` and ```/cmd_vel```. 

In [ ]:

c1 = time.time() #offset of timer 
time1 = list(range(0, 10))
odom_linear =list()
cmdvel_linear=list()

In [ ]:
def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    return [line1,line2]

#odom clbk
def clb_odom1(msg):
    if len(time1)>10:
      time1.pop(0)
     
    if len(odom_linear)>10:
      odom_linear.pop(0)
    ip = np.float32(msg.twist.twist.linear.x)
    iy = np.float32(time.time()-c1)
    #data update
    odom_linear.append(iy)
    time1.append(ix)
    
   

jr.subscribe('/odom',Odometry, clb_odom1)
    

> Remark: subscribers have been put in separate cells for avoiding conflicts (i.e overwriting)

In [ ]:
def clb_cmdvel1(msg):
    if len(cmdvel_linear)>10:
      cmdvel_linear.pop(0)
    iy = np.float32(msg.linear.x)
    cmdvel_linear.append(iy)
    

jr.subscribe('/cmd_vel', Twist, clb_cmdvel1)

In [ ]:
fig1, ax1 = plt.subplots()



line1, = ax1.plot([], [], color ='r', label ='/odom')
line2, = ax1.plot([], [], color ='b', label ='/cmd_vel')
ax1.set_xlabel('time (s)') 
ax1.set_ylabel('velocity m/s')
ax1.set_title("Linear velocity")
plt.legend(loc='upper right')

In [ ]:
def animate1(i):
    #refresh of line
    line1.set_data(time1, odom_linear)
    line2.set_data(time1, cmdvel_linear)
    ax1.axis([time1[0]-0.2,time1[9]+0.2,-0.5,0.5])    
    return [line1,line2]

#animator
anim1 = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

## Angular Velocity

In the  figure below, it is possible to monitor the **angular velocity** subscribed by ```/odom```and``` /cmd_vel```. 
> The same line of conduct used for the aforementioned linear velocity is here now adopted 

In [ ]:
clock2 = time.time() 
time2 = list(range(0, 10))
odom_angular =list()
cmdvel_angular=list()

In [ ]:
def init2():
    line3.set_data([], [])
    line4.set_data([], [])
    return [line3,line4]

def clb_odom2(msg):
    if len(time2)>10:
      time2.pop(0)
     
    if len(odom_angular)>10:
      odom_angular.pop(0)
    iupsilon = np.float32(msg.twist.twist.angular.z)
    ix = np.float32(time.time()-clock2)
    odom_angular.append(iupsilon)
    time2.append(ix)
    
   

jr.subscribe('/odom',Odometry, clb_odom2)

In [ ]:
def clb_cmdvel2(msg):
    if len(cmdvel_angular)>10:
      cmdvel_angular.pop(0)
    iupsilon = np.float32(msg.angular.z)
    cmdvel_angular.append(iupsilon)
    

jr.subscribe('/cmd_vel', Twist, clb_cmdvel2)

In [ ]:
fig2, ax2 = plt.subplots()



line3, = ax2.plot([], [], color ='r', label ='/odom')
line4, = ax2.plot([], [], color ='b', label ='/cmd_vel')
ax2.set_xlabel('time (s)') 
ax2.set_ylabel('velocity rad/s')
ax2.set_title("Angular velocity")
plt.legend(loc='upper right')

In [ ]:

def animate2(i):
    line3.set_data(time2, odom_angular)
    line4.set_data(time2, cmdvel_angular)
    ax2.axis([time2[0]-0.2,time2[9]+0.2,-0.5,0.5])
    return [line3,line4]

anim2 = animation.FuncAnimation(fig2, animate2, init_func=init2,
                               frames=100, interval=20, blit=True)

## Show Position

In the figure below, I iterated the same method as before, showing the position of the robot inside the environment. The values are retrieved from the ```odometry``` callback  and then they have been shown on (x,y) plane.

In [ ]:
x_data=list()
y_data=list()

In [ ]:
fig, ax = plt.subplots()

ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))
ax.set_xlabel('x') 
ax.set_ylabel('y')
ax.set_title("Position")
line, = ax.plot([], [], 'ro')
    

In [ ]:
def init():
    line.set_data([], [])
    return (line,)


    
def odom_callback(msg):
    if len(x_data)>50:
      x_data.pop(0)
      
    if len(y_data)>50:
      y_data.pop(0)
    
    iupsilon = np.float32(msg.pose.pose.position.y)
    ix = np.float32(msg.pose.pose.position.x)
    y_data.append(iupsilon)
    x_data.append(ix)
    

    



jr.subscribe('/odom', Odometry, odom_callback)



In [ ]:
def animate(i):
    line.set_data(x_data, y_data)
    return (line,)

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

## Show Orientation

To conclude with, the orientation of the robot could be visualized, by showing an arrow whose:
- tail is positioned where the robot is located (inside the virtual environment)
- orientation is the same of the robot

In [ ]:
fig4, ax4 = plt.subplots()

ax4.quiver(0,0,dx,dy,label='Orientation')
ax4.set_title("Orientation")
ax4.legend()
line4, = ax.plot([], [], 'ro')

In [ ]:
def init4():
    ax4.arrow(0,0,dx,dy,label='Orientation')
    return (line4,)

def animate4(i):
    ax4.clear()
    ax4.quiver(x_data[48],y_data[48],x_data[49]-x_data[48],y_data[49]-y_data[48],label='Orientation')
    ax4.legend()
    ax4.set_title("Orientation")
    return (line4,)

anim4=animation.FuncAnimation(fig4, animate4, init_func=init4, frames=1, interval=100, blit=True)